In [4]:
suppressPackageStartupMessages({

    library(tidyverse)
    library(here)
    library(bdkn)
    library(VariantAnnotation)

})

In [5]:
vcf <- readVcf(here("data/vcf/merged_full.vcf.gz"))

In [6]:
gt <- geno(vcf)$GT %>% replace(. == ".", NA)

In [7]:
mode(gt) <- "numeric"

In [8]:
gt <- as_tibble(gt)

In [9]:
gt %>% head

Chimp,spy1,mez2,denisova8,S_French-1,S_Sardinian-1,S_Han-2,S_Dai-2,S_Papuan-2,S_Karitiana-1,⋯,S_Mbuti-1,S_Yoruba-2,S_Mandenka-1,ustishim,a00_1,a00_2,kk1,mota,bichon,loschbour
NA,NA,NA,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
NA,NA,NA,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
NA,NA,NA,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
NA,NA,NA,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
0,NA,NA,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
0,NA,NA,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0


In [10]:
nrow(gt)

[1] 6912056

In [22]:
gt <- gt %>% dplyr::select(-c(spy1, mez2, denisova8, kk1, mota, bichon, loschbour)) %>% .[complete.cases(.), ]

In [25]:
counts <- list(
    a = sum((gt$ustishim == 0) & (gt$Chimp == gt$a00_1) & (gt$Chimp != 0), na.rm = TRUE),
    d = sum((gt$Chimp != 0) & (gt$a00_1 != 0) & (gt$ustishim != 0), na.rm = TRUE),
    e = sum((gt$Chimp == 0) & (gt$a00_1 == 0) & (gt$ustishim == 1), na.rm = TRUE),
    f = sum((gt$Chimp == 0) & (gt$a00_1 == 1) & (gt$ustishim == 0), na.rm = TRUE)
)
counts

$a
[1] 876

$d
[1] 202

$e
[1] 21

$f
[1] 1016

In [23]:
counts <- list(
    a = sum((gt$ustishim == 0) & (gt$Chimp == gt$a00_2) & (gt$Chimp != 0), na.rm = TRUE),
    d = sum((gt$Chimp != 0) & (gt$a00_2 != 0) & (gt$ustishim != 0), na.rm = TRUE),
    e = sum((gt$Chimp == 0) & (gt$a00_2 == 0) & (gt$ustishim == 1), na.rm = TRUE),
    f = sum((gt$Chimp == 0) & (gt$a00_2 == 1) & (gt$ustishim == 0), na.rm = TRUE)
)
counts

$a
[1] 875

$d
[1] 202

$e
[1] 21

$f
[1] 962

In [15]:
mutation_rate <- 7.4e-10

In [16]:
(counts$a + counts$d) / (nrow(gt) * mutation_rate)

[1] 249661.7

In [24]:
counts$f / (nrow(gt) * mutation_rate)

[1] 241731.6